In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scripts.data_import import data_import
from scripts.run_optimization_model import run_model

# Read Data

In [2]:
file = 'data_s3'
path = 'data/' + file + '.xlsx'
data, basis, bess, service, revenue_change = data_import(path)

/Users/angelmah/Documents/codes/bess-optimizer/handover/scripts/data_import.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['basis']  = data['basis'].dropna().drop(['Tag']).replace({'YES': 1, 'NO': 0})
/Users/angelmah/Documents/codes/bess-optimizer/handover/scripts/data_import.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['bess']  = data['bess'].dropna().drop(['Tag', np.nan]).replace({'YES': 1, 'NO': 0})


# Base case simulation

In [3]:
# Override Excel settings (if required)
# bess['cap_power'] = 1
# bess['cap_energy'] = 4
bess['s_eff'] = 1

# Model optimality gap and time limits, solving process will stop when any of the conditions are satisfied
model_optimality_gap = 0.005     # proximity to optimal solution, 0.01 means 1% difference with the optimal solution
model_time_limit = 100          # seconds

# Result file name
result_html_name = "result"
result_html_path = "results/" + result_html_name

# Run model
packaged_data = [data, basis, bess, service] 
annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_path, model_optimality_gap = model_optimality_gap, model_time_limit = model_time_limit)
[NPV, ROI, IRR, BCR, PP] = financial_metrics


HTML file generated:results/result.html


# Sensitivity analysis

In [4]:
power_range = list(range(1, 16))
energy_range = list(range(1, 16))

econs_sens_keys = []
econs_sens_values = []

for power in power_range:
    for energy in energy_range:

        power_target = power * 0.5
        energy_target = energy * 0.5

        print("Solving: ", power_target, energy_target)

        bess['cap_power'] = power_target
        bess['cap_energy'] = energy_target

        packaged_data = [data, basis, bess, service] 
        annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_name = [])
        [NPV, ROI, IRR, BCR, PP] = financial_metrics

        econs_sens_keys.append([power_target, energy_target])
        econs_sens_values.append([annual_revenue] + financial_metrics)

df = pd.DataFrame(index = pd.MultiIndex.from_tuples(econs_sens_keys, names = ['Power', 'Energy']), data = econs_sens_values, columns = ['annual_revenue', 'NPV', 'ROI', 'IRR', 'BCR', 'PP'])
df['annual_revenue_per_kw'] = df['annual_revenue']/df.index.get_level_values(0)/1000
df = df.reset_index()
df

# Export result to excel
df.to_excel("results/sens_result.xlsx")

Solving:  0.5 0.5
Solving:  0.5 1.0
Solving:  0.5 1.5
Solving:  0.5 2.0
Solving:  0.5 2.5
Solving:  0.5 3.0
Solving:  0.5 3.5
Solving:  0.5 4.0
Solving:  0.5 4.5
Solving:  0.5 5.0
Solving:  0.5 5.5
Solving:  0.5 6.0
Solving:  0.5 6.5
Solving:  0.5 7.0
Solving:  0.5 7.5
Solving:  1.0 0.5
Solving:  1.0 1.0
Solving:  1.0 1.5
Solving:  1.0 2.0
Solving:  1.0 2.5
Solving:  1.0 3.0
Solving:  1.0 3.5
Solving:  1.0 4.0
Solving:  1.0 4.5
Solving:  1.0 5.0
Solving:  1.0 5.5
Solving:  1.0 6.0
Solving:  1.0 6.5
Solving:  1.0 7.0
Solving:  1.0 7.5
Solving:  1.5 0.5
Solving:  1.5 1.0
Solving:  1.5 1.5
Solving:  1.5 2.0
Solving:  1.5 2.5
Solving:  1.5 3.0
Solving:  1.5 3.5
Solving:  1.5 4.0
Solving:  1.5 4.5
Solving:  1.5 5.0
Solving:  1.5 5.5
Solving:  1.5 6.0
Solving:  1.5 6.5
Solving:  1.5 7.0
Solving:  1.5 7.5
Solving:  2.0 0.5
Solving:  2.0 1.0
Solving:  2.0 1.5
Solving:  2.0 2.0
Solving:  2.0 2.5
Solving:  2.0 3.0
Solving:  2.0 3.5
Solving:  2.0 4.0
Solving:  2.0 4.5
Solving:  2.0 5.0
Solving:  

KeyboardInterrupt: 

In [ ]:
# Calculating Energy/Power ratio for the contour plot
df['Ratio'] = df['Energy'] / df['Power']
pivot_df = df.pivot(index="Energy", columns="Power", values="NPV") * 100
pivot_ratio = df.pivot(index="Energy", columns="Power", values="Ratio")

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap with contour lines
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  # High values red, low values blue
    colorbar=dict(title='ROI (%)'),
    zmin=np.min(z),  # Define the range of z for better contrast
    zmax=np.max(z)
))

# Add contour lines for the Energy/Power ratio with text labels
fig.add_trace(go.Contour(
    z=pivot_ratio.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    showscale=False,  # Hide the scale for contour lines
    contours=dict(
        coloring='lines',
        showlabels=True,  # Enable labels on contours
        labelfont=dict(size=12, color='black'),  # Customize the font of the labels
        start=0.5,  # Starting value for contour lines
        end=4,     # Ending value for contour lines
        size=0.5   # Interval between contour lines
    ),
    line=dict(color='black', width=2),
    # ncontours=50  # Increase the number of contour lines

))

# Adding titles and labels, and setting axis ranges to start from 1
fig.update_layout(
    title='Heatmap of ROI with Energy/Power Ratio Contour Lines',
    xaxis=dict(title='Power', range=[0.5, 8]),
    yaxis=dict(title='Energy', range=[0.5, 8])
)

# Display the figure
fig.show()
